In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from mus_align.mesa13 import load_mesa13_alignment, load_computed_alignment, load_computed_alignment_baseline

In [3]:
from mus_align.types import *
from mus_align.eval import evaluate

In [4]:
! pip install tqdm
from tqdm import tqdm

In [15]:
import os
import pickle
import numpy as np

In [6]:
tags = ["andante_con_moto", "lux_aeterna", "maple_leaf_rag", "pink_chinese_rag", "prelude_in_c", "the_maid",
       "omaggio_a_brahms", "promenade", "a_lesson", "liebestraum", "berceuse", "gymnopedie", "pastorale"]

In [7]:
repeats = ["maple_leaf_rag", "a_lesson"]
no_repeats = ["andante_con_moto", "lux_aeterna", "pink_chinese_rag", "prelude_in_c", "the_maid",
       "omaggio_a_brahms", "promenade", "liebestraum", "berceuse", "gymnopedie", "pastorale"]

### Align and evaluate

In [48]:
os.makedirs('alignments/mesa13', exist_ok=True)

In [49]:
path = 'alignments/mesa13/strict'
os.makedirs(path, exist_ok=True)
for tag in tqdm(tags):
    aligned_score = load_computed_alignment(tag=tag, from_precomputed=False, use_repeats=False, use_measures=False, use_staff_metadata=False)
    with open(f'{path}/{tag}_aligned_score.zip', 'wb') as f:
        f.write(AlignedScore.as_project_zip(aligned_score))

In [67]:
path = 'alignments/mesa13/jltr'
os.makedirs(path, exist_ok=True)
for tag in tqdm(tags):
    aligned_score = load_computed_alignment(tag=tag, from_precomputed=False, use_repeats=True, use_measures=False, use_staff_metadata=False)
    with open(f'{path}/{tag}_aligned_score.zip', 'wb') as f:
        f.write(AlignedScore.as_project_zip(aligned_score))

In [50]:
path = 'alignments/mesa13/weak'
os.makedirs(path, exist_ok=True)
for tag in tqdm(tags):
    aligned_score = load_computed_alignment(tag=tag, from_precomputed=False, use_repeats=True, use_measures=True, use_staff_metadata=False)
    with open(f'{path}/{tag}_aligned_score.zip', 'wb') as f:
        f.write(AlignedScore.as_project_zip(aligned_score))

In [51]:
path = 'alignments/mesa13/strong'
os.makedirs(path, exist_ok=True)
for tag in tqdm(tags):
    aligned_score = load_computed_alignment(tag=tag, from_precomputed=False, use_repeats=True, use_measures=True, use_staff_metadata=True)
    with open(f'{path}/{tag}_aligned_score.zip', 'wb') as f:
        f.write(AlignedScore.as_project_zip(aligned_score))

In [52]:
path = 'alignments/mesa13/baseline'
os.makedirs(path, exist_ok=True)
for tag in tqdm(tags):
    aligned_score = load_computed_alignment_baseline(tag=tag, from_precomputed=False)
    with open(f'{path}/{tag}_aligned_score.zip', 'wb') as f:
        f.write(AlignedScore.as_project_zip(aligned_score))

In [64]:
accs = {'strict': {}, 'jltr': {}, 'weak': {}, 'strong': {}, 'baseline': {}}

In [65]:
for tag in tqdm(tags):
    gt = load_mesa13_alignment(tag=tag)
    with open(f'alignments/mesa13/strict/{tag}_aligned_score.zip', 'rb') as f:
        aligned_score_strict = AlignedScore.from_project_zip(f.read())
    with open(f'alignments/mesa13/jltr/{tag}_aligned_score.zip', 'rb') as f:
        aligned_score_jltr = AlignedScore.from_project_zip(f.read())
    with open(f'alignments/mesa13/weak/{tag}_aligned_score.zip', 'rb') as f:
        aligned_score_weak = AlignedScore.from_project_zip(f.read())
    with open(f'alignments/mesa13/strong/{tag}_aligned_score.zip', 'rb') as f:
        aligned_score_strong = AlignedScore.from_project_zip(f.read())
    with open(f'alignments/mesa13/baseline/{tag}_aligned_score.zip', 'rb') as f:
        aligned_score_baseline = AlignedScore.from_project_zip(f.read())
    accs['strict'][tag] = evaluate(aligned_score_strict, gt)
    accs['jltr'][tag] = evaluate(aligned_score_jltr, gt)
    accs['weak'][tag] = evaluate(aligned_score_weak, gt)
    accs['strong'][tag] = evaluate(aligned_score_strong, gt)
    accs['baseline'][tag] = evaluate(aligned_score_baseline, gt)

100%|███████████████████████████████████████████| 13/13 [02:31<00:00, 11.69s/it]


In [66]:
for m in [tags, repeats, no_repeats]:
    print('baseline', round(np.mean([accs['baseline'][k][0] for k in m]), 2), round(np.mean([np.mean(accs['baseline'][k][1]) for k in m]), 1), 
          round(np.std([np.mean(accs['baseline'][k][1]) for k in m]),1))
    print('strict', round(np.mean([accs['strict'][k][0] for k in m]), 2), round(np.mean([np.mean(accs['strict'][k][1]) for k in m]), 1), 
          round(np.std([np.mean(accs['strict'][k][1]) for k in m]),1))
    print('jltr', round(np.mean([accs['jltr'][k][0] for k in m]), 2), round(np.mean([np.mean(accs['jltr'][k][1]) for k in m]), 1), 
          round(np.std([np.mean(accs['jltr'][k][1]) for k in m]),1))
    print('weak', round(np.mean([accs['weak'][k][0] for k in m]), 2), round(np.mean([np.mean(accs['weak'][k][1]) for k in m]), 1), 
          round(np.std([np.mean(accs['weak'][k][1]) for k in m]),1))
    print('strong', round(np.mean([accs['strong'][k][0] for k in m]), 2), round(np.mean([np.mean(accs['strong'][k][1]) for k in m]), 1), 
          round(np.std([np.mean(accs['strong'][k][1]) for k in m]),1))
    print('--')

baseline 0.33 10.9 11.6
strict 0.72 1.9 3.7
jltr 0.82 0.4 0.2
weak 0.86 0.4 0.2
strong 0.88 0.3 0.2
--
baseline 0.17 23.6 10.2
strict 0.2 10.0 3.0
jltr 0.83 0.3 0.0
weak 0.93 0.2 0.0
strong 0.95 0.2 0.0
--
baseline 0.36 8.6 10.2
strict 0.82 0.4 0.2
jltr 0.82 0.4 0.2
weak 0.85 0.4 0.3
strong 0.87 0.3 0.2
--


### Evaluate from precomputed

In [8]:
accs = {'strict': {}, 'jltr': {}, 'weak': {}, 'strong': {}, 'baseline': {}}

In [18]:
for tag in tqdm(tags):
    gt = load_mesa13_alignment(tag=tag)
    for setting in accs.keys():
        with open(f"alignments/mesa13/{setting}/{tag}_aligned_score.zip", "rb") as f:
            aligned_score = AlignedScore.from_project_zip(f.read())
        accs[setting][tag] = evaluate(aligned_score, gt)

100%|███████████████████████████████████████████| 13/13 [02:08<00:00,  9.90s/it]


In [19]:
for m in [tags, repeats, no_repeats]:
    print('baseline', round(np.mean([accs['baseline'][k][0] for k in m]), 2), round(np.mean([np.mean(accs['baseline'][k][1]) for k in m]), 1), 
          round(np.std([np.mean(accs['baseline'][k][1]) for k in m]),1))
    print('strict', round(np.mean([accs['strict'][k][0] for k in m]), 2), round(np.mean([np.mean(accs['strict'][k][1]) for k in m]), 1), 
          round(np.std([np.mean(accs['strict'][k][1]) for k in m]),1))
    print('jltr', round(np.mean([accs['jltr'][k][0] for k in m]), 2), round(np.mean([np.mean(accs['jltr'][k][1]) for k in m]), 1), 
          round(np.std([np.mean(accs['jltr'][k][1]) for k in m]),1))
    print('weak', round(np.mean([accs['weak'][k][0] for k in m]), 2), round(np.mean([np.mean(accs['weak'][k][1]) for k in m]), 1), 
          round(np.std([np.mean(accs['weak'][k][1]) for k in m]),1))
    print('strong', round(np.mean([accs['strong'][k][0] for k in m]), 2), round(np.mean([np.mean(accs['strong'][k][1]) for k in m]), 1), 
          round(np.std([np.mean(accs['strong'][k][1]) for k in m]),1))
    print('--')

baseline 0.33 10.9 11.6
strict 0.72 1.9 3.7
jltr 0.82 0.4 0.2
weak 0.86 0.4 0.2
strong 0.88 0.3 0.2
--
baseline 0.17 23.6 10.2
strict 0.2 10.0 3.0
jltr 0.83 0.3 0.0
weak 0.93 0.2 0.0
strong 0.95 0.2 0.0
--
baseline 0.36 8.6 10.2
strict 0.82 0.4 0.2
jltr 0.82 0.4 0.2
weak 0.85 0.4 0.3
strong 0.87 0.3 0.2
--
